In [21]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from scipy.spatial import HalfspaceIntersection, ConvexHull
from itertools import combinations
import networkx as nx

## 4.5 Identifying dominated strategies

### 4.5.1 Domination by pure strategies

To check whether a given strategy $s_1$ is dominated (strictly) by some pure strategy is quite simple. We just need to check if there is a pure action $a_i$ we can take, such that $a_i$ is better than $s_1$ for all of the opponent's pure actions $a_j$.

Note that domination only depends on your own payoff, not that of any other player, so we can remove player 2 from payoff tables for simplicity.

Now consider:

$
\begin{array}{c|cc}
\text{} & \text{0} & \text{1} & \text{2} \\
\hline
\text{0} & 4 & 3 & 2 \\
\text{1} & 1 & 2 & 1 \\
\end{array}
$

We will check whether the row player's strategy of $[\frac{1}{2},\frac{1}{2}]$ is dominated.

In [46]:
payoff = np.array([
    [4,3,2],
    [1,2,1]
])

s1 = np.array([0.5,0.5])
for A1_i in range(2):
    dominated = True
    for A2_j in range(3):
        if s1.dot(payoff[:,A2_j])>=payoff[A1_i,A2_j]:
            dominated = False
            break
    if dominated:
        print("dominated by",A1_i)
        break

dominated by 0


And it is! Because unsuprisingly, it is always better to go 0 than 1, so any mixed strategy is dominated by the pure one.

You might think that we have neglected to check the mixed strategies of the opponent above. What if there was a mixed strategy for the opponent, such that $s_1$ is actually as good as $a_i$? Fortunately, we don't have to check this! The utility of $s_1$ is just $s_1U_1$ (a vector with a value for each of the opponent's actions). The value of $a_i$ is just $U_1^i$ (i'th row in $U_1$). If $a_i$ is better than $s_i$ for all the opponent's pure strategies then each entry must be greater. So any mixture is greater. From above, we get:

$
\begin{array}{c|ccc}
\text{value of} & \text{0} & \text{1} & \text{2} \\
\hline
a_0 & 4 & 3 & 2 \\
s_1 & 2.5 & 2.5 & 1.5
\end{array}
$

Any mixing of the columns won't matter.

### 4.5.2 Domination by mixed strategies

Things get more complicated if we want to check if our strategy can be dominated by a mixed strategy. Consider the following linear program:

$$
\begin{align*}
\text{minimise} \quad & \sum_k p_k \\
\text{subject to} \quad & \sum_{k} p_k U_1(A_1^k,A_2^j) \geq U_1(s_1,A_2^j) \quad \forall A_2^j \\
\text{} \quad & p_k \geq 0 \\
\end{align*}
$$

If this linear program finds $p$ such that $\sum_k p_k<1$ then we know we have a mixed strategy that can dominate $s_1$. Why? If the sum is less than 1, then we can make the sum equal to 1 by increasing at least one $p_k$. If we do this then we know we will be improving on $U_1(s_1,A_2^j)$ in at least one place. Note: This method requires the utility to be scaled so all are greater than 0.

Consider these payoffs:

$
\begin{array}{c|cc}
\text{} & \text{X} & \text{Y} & \text{Z} \\
\hline
\text{X} & 3 & 1 & 2 \\
\text{Y} & 1 & 2 & 1
\end{array}
$

Is there a strategy that beats $[0.5,0.25,0.25]$?

In [50]:
old_solution = np.array([0.5,0.25,0.25])
c = np.array([1,1,1]) # minimise the sum of these
A = np.array([
    [3,1,2],
    [1,2,1]
])
b = A.dot(old_solution)
res = linprog(c, A_ub=-A, b_ub=-b)
print("function:",res["fun"])
print("new solution:",(res["x"]/np.sum(res["x"])).round(2))

function: 0.95
new solution: [0.68 0.32 0.  ]


This makes sense. $Z$ is not as good as going $X$, so it's better to put more probability mass in $X$.  

### 4.5.3 Iterated dominance

At this point we can consider the process of eliminating pure strategies. This is naturally something we might want to do in order to reduce our search space. A pure strategy $a_i$ (and any mixed strategies using it) can be eliminated if there is a mixed strategy which dominates it without using it. Any strategy which did use $a_i$ could be improved by replacing it's probability density with the dominating mixed strategy. I.e., if
$$u_1 < w_2u_2+w_3u_3+w_4u_4$$
then 
$$p_1u_1+p_2u_2+p_3u_3+p_4u_4<p_2(1+p_1w_2)u_2+p_3(1+p_1w_3)u_3+p_4(1+p_1w_4)u_4$$

So we can look to remove pure strategies in this manner. Take the following example:

$
\begin{array}{c|cc}
\text{} & \text{X} & \text{Y} & \text{Z} \\
\hline
\text{X} & 3,1 & 0,1 & 0,0 \\
\text{Y} & 1,1 & 1,1 & 5,0 \\
\text{Z} & 0,1 & 4,1 & 0,0
\end{array}
$

In [65]:
def check_pure_strategy_is_dominated(U,row):
    c = np.ones(U.shape[0]-1)
    A = U[np.arange(U.shape[0]) != row].T
    b = U[row]
    res = linprog(c, A_ub=-A, b_ub=-b)
    if not res["fun"] is None:
        if res["fun"]<1:
            return True
    return False

U1 = np.array([[3,0,0],[1,1,5],[0,4,0]])
U2 = np.array([[1,1,0],[1,1,0],[1,1,0]])
for i in range(10):
    rows,cols = U1.shape
    any_removed = False
    for row in range(rows):
        if check_pure_strategy_is_dominated(U1,row):
            U1 = U1[np.arange(rows) != row]
            U2 = U2[np.arange(rows) != row]
            any_removed = True
            print("row",row+1,"dominated")
            break

    if not any_removed:
        for col in range(cols):
            if check_pure_strategy_is_dominated(U2.T,col):
                U1 = U1[:,np.arange(cols) != col]
                U2 = U2[:,np.arange(cols) != col]
                any_removed = True
                print("col",col+1,"dominated")
                break
                
    if not any_removed:
        break

col 3 dominated
row 2 dominated


What happens is col 3 is removed, as $X$ and $Y$ are always better choices for player 2. Then the game becomes:

$
\begin{array}{c|cc}
\text{} & \text{X} & \text{Y} \\
\hline
\text{X} & 3,1 & 0,1 \\
\text{Y} & 1,1 & 1,1 \\
\text{Z} & 0,1 & 4,1
\end{array}
$

At this point it's then clear that row 2 can be removed, as a 50:50 strategy between $X$ and $Z$ is better than $Y$, which gives:

$
\begin{array}{c|cc}
\text{} & \text{X} & \text{Y} \\
\hline
\text{X} & 3,1 & 0,1 \\
\text{Z} & 0,1 & 4,1
\end{array}
$